# Your First RAG Application

In this notebook, we'll walk you through each of the components that are involved in a simple RAG application.

We won't be leveraging any fancy tools, just the OpenAI Python SDK, Numpy, and some classic Python.

> NOTE: This was done with Python 3.12.3.

> NOTE: There might be [compatibility issues](https://github.com/wandb/wandb/issues/7683) if you're on NVIDIA driver >552.44 As an interim solution - you can rollback your drivers to the 552.44.

## Table of Contents:

- Task 1: Imports and Utilities
- Task 2: Documents
- Task 3: Embeddings and Vectors
- Task 4: Prompts
- Task 5: Retrieval Augmented Generation
  - 🚧 Activity #1: Augment RAG

Let's look at a rather complicated looking visual representation of a basic RAG application.

<img src="https://i.imgur.com/vD8b016.png" />

## Task 1: Imports and Utility

We're just doing some imports and enabling `async` to work within the Jupyter environment here, nothing too crazy!

In [ ]:
from aimakerspace.text_utils import TextFileLoader, CharacterTextSplitter
from aimakerspace.vectordatabase import VectorDatabase
import asyncio

In [ ]:
import nest_asyncio
nest_asyncio.apply()

## Task 2: Documents

We'll be concerning ourselves with this part of the flow in the following section:

<img src="https://i.imgur.com/jTm9gjk.png" />

### Loading Source Documents

So, first things first, we need some documents to work with.

While we could work directly with the `.txt` files (or whatever file-types you wanted to extend this to) we can instead do some batch processing of those documents at the beginning in order to store them in a more machine compatible format.

In this case, we're going to parse our text file into a single document in memory.

Let's look at the relevant bits of the `TextFileLoader` class:

```python
def load_file(self):
        with open(self.path, "r", encoding=self.encoding) as f:
            self.documents.append(f.read())
```

We're simply loading the document using the built in `open` method, and storing that output in our `self.documents` list.

> NOTE: We're using blogs from PMarca (Marc Andreessen) as our sample data. This data is largely irrelevant as we want to focus on the mechanisms of RAG, which includes out data's shape and quality - but not specifically what the contents of the data are. 


In [ ]:
text_loader = TextFileLoader("data/PMarcaBlogs.txt")
documents = text_loader.load_documents()
len(documents)

In [ ]:
print(documents[0][:100])

### Splitting Text Into Chunks

As we can see, there is one massive document.

We'll want to chunk the document into smaller parts so it's easier to pass the most relevant snippets to the LLM.

There is no fixed way to split/chunk documents - and you'll need to rely on some intuition as well as knowing your data *very* well in order to build the most robust system.

For this toy example, we'll just split blindly on length.

>There's an opportunity to clear up some terminology here, for this course we will be stick to the following:
>
>- "source documents" : The `.txt`, `.pdf`, `.html`, ..., files that make up the files and information we start with in its raw format
>- "document(s)" : single (or more) text object(s)
>- "corpus" : the combination of all of our documents

As you can imagine (though it's not specifically true in this toy example) the idea of splitting documents is to break them into managable sized chunks that retain the most relevant local context.

In [ ]:
text_splitter = CharacterTextSplitter()
split_documents = text_splitter.split_texts(documents)
len(split_documents)

Let's take a look at some of the documents we've managed to split.

In [ ]:
split_documents[0:1]

## Task 3: Embeddings and Vectors

Next, we have to convert our corpus into a "machine readable" format as we explored in the Embedding Primer notebook.

Today, we're going to talk about the actual process of creating, and then storing, these embeddings, and how we can leverage that to intelligently add context to our queries.

### OpenAI API Key

In order to access OpenAI's APIs, we'll need to provide our OpenAI API Key!

You can work through the folder "OpenAI API Key Setup" for more information on this process if you don't already have an API Key!

In [ ]:
import os
import openai
from dotenv import load_dotenv


# Load environment variables from .env file
load_dotenv()

# Get API key from environment variables
api_key = os.getenv("OPENAI_API_KEY")

# Set the API key for OpenAI
openai.api_key = api_key
os.environ["OPENAI_API_KEY"] = api_key
print("✅ OpenAI API key set successfully")


In [ ]:
# Test that the API key is working
import os
print("🔑 API Key Status:")
print(f"Environment variable set: {'OPENAI_API_KEY' in os.environ}")
print(f"OpenAI API key set: {hasattr(openai, 'api_key') and openai.api_key is not None}")

if 'OPENAI_API_KEY' in os.environ:
    print(f"Key starts with: {os.environ['OPENAI_API_KEY'][:10]}...")
    print("✅ Ready to use OpenAI APIs!")
else:
    print("❌ API key not set properly")


### Vector Database

Let's set up our vector database to hold all our documents and their embeddings!

While this is all baked into 1 call - we can look at some of the code that powers this process to get a better understanding:

Let's look at our `VectorDatabase().__init__()`:

```python
def __init__(self, embedding_model: EmbeddingModel = None):
        self.vectors = defaultdict(np.array)
        self.embedding_model = embedding_model or EmbeddingModel()
```

As you can see - our vectors are merely stored as a dictionary of `np.array` objects.

Secondly, our `VectorDatabase()` has a default `EmbeddingModel()` which is a wrapper for OpenAI's `text-embedding-3-small` model.

> **Quick Info About `text-embedding-3-small`**:
> - It has a context window of **8191** tokens
> - It returns vectors with dimension **1536**

#### ❓Question #1:

The default embedding dimension of `text-embedding-3-small` is 1536, as noted above. 

1. Is there any way to modify this dimension? 
2. What technique does OpenAI use to achieve this? 

> NOTE: Check out this [API documentation](https://platform.openai.com/docs/api-reference/embeddings/create) for the answer to question #1.1, and [this documentation](https://platform.openai.com/docs/guides/embeddings/use-cases) for an answer to question #1.2!


##### ✅ Answer:
1. Yes there is, one can modify the dimensions parameter in the embedding vector.
2. The embedding vector can be shortened without changing the meaning of it's concept representing properties. An example of this is on the MTEB benchmark, a text-embedding-3-large embedding can be shortened to a size of 256 while still outperforming an unshortened text-embedding-ada-002 embedding with a size of 1536. Shortening the dimensions parameter trades off some accuracy in exchange for smaller vector size and cost.

We can call the `async_get_embeddings` method of our `EmbeddingModel()` on a list of `str` and receive a list of `float` back!

```python
async def async_get_embeddings(self, list_of_text: List[str]) -> List[List[float]]:
        return await aget_embeddings(
            list_of_text=list_of_text, engine=self.embeddings_model_name
        )
```

We cast those to `np.array` when we build our `VectorDatabase()`:

```python
async def abuild_from_list(self, list_of_text: List[str]) -> "VectorDatabase":
        embeddings = await self.embedding_model.async_get_embeddings(list_of_text)
        for text, embedding in zip(list_of_text, embeddings):
            self.insert(text, np.array(embedding))
        return self
```

And that's all we need to do!

In [ ]:
vector_db = VectorDatabase()
vector_db = asyncio.run(vector_db.abuild_from_list(split_documents))

#### ❓Question #2:

What are the benefits of using an `async` approach to collecting our embeddings?

> NOTE: Determining the core difference between `async` and `sync` will be useful! If you get stuck - ask ChatGPT!

##### ✅ Answer: Sync waits for each embedding to finish before starting the next (blocking, simple, lower throughput); async launches multiple requests and processes results as they complete (non-blocking, higher throughput), best for I/O-bound, high-latency APIs with bounded concurrency and retries.


So, to review what we've done so far in natural language:

1. We load source documents
2. We split those source documents into smaller chunks (documents)
3. We send each of those documents to the `text-embedding-3-small` OpenAI API endpoint
4. We store each of the text representations with the vector representations as keys/values in a dictionary

### Semantic Similarity

The next step is to be able to query our `VectorDatabase()` with a `str` and have it return to us vectors and text that is most relevant from our corpus.

We're going to use the following process to achieve this in our toy example:

1. We need to embed our query with the same `EmbeddingModel()` as we used to construct our `VectorDatabase()`
2. We loop through every vector in our `VectorDatabase()` and use a distance measure to compare how related they are
3. We return a list of the top `k` closest vectors, with their text representations

There's some very heavy optimization that can be done at each of these steps - but let's just focus on the basic pattern in this notebook.

> We are using [cosine similarity](https://www.engati.com/glossary/cosine-similarity) as a distance metric in this example - but there are many many distance metrics you could use - like [these](https://flavien-vidal.medium.com/similarity-distances-for-natural-language-processing-16f63cd5ba55)

> We are using a rather inefficient way of calculating relative distance between the query vector and all other vectors - there are more advanced approaches that are much more efficient, like [ANN](https://towardsdatascience.com/comprehensive-guide-to-approximate-nearest-neighbors-algorithms-8b94f057d6b6)

In [ ]:
vector_db.search_by_text("What is the Michael Eisner Memorial Weak Executive Problem?", k=3)

## Task 4: Prompts

In the following section, we'll be looking at the role of prompts - and how they help us to guide our application in the right direction.

In this notebook, we're going to rely on the idea of "zero-shot in-context learning".

This is a lot of words to say: "We will ask it to perform our desired task in the prompt, and provide no examples."

### XYZRolePrompt

Before we do that, let's stop and think a bit about how OpenAI's chat models work.

We know they have roles - as is indicated in the following API [documentation](https://platform.openai.com/docs/api-reference/chat/create#chat/create-messages)

There are three roles, and they function as follows (taken directly from [OpenAI](https://platform.openai.com/docs/guides/gpt/chat-completions-api)):

- `{"role" : "system"}` : The system message helps set the behavior of the assistant. For example, you can modify the personality of the assistant or provide specific instructions about how it should behave throughout the conversation. However note that the system message is optional and the model’s behavior without a system message is likely to be similar to using a generic message such as "You are a helpful assistant."
- `{"role" : "user"}` : The user messages provide requests or comments for the assistant to respond to.
- `{"role" : "assistant"}` : Assistant messages store previous assistant responses, but can also be written by you to give examples of desired behavior.

The main idea is this:

1. You start with a system message that outlines how the LLM should respond, what kind of behaviours you can expect from it, and more
2. Then, you can provide a few examples in the form of "assistant"/"user" pairs
3. Then, you prompt the model with the true "user" message.

In this example, we'll be forgoing the 2nd step for simplicities sake.

#### Utility Functions

You'll notice that we're using some utility functions from the `aimakerspace` module - let's take a peek at these and see what they're doing!

##### XYZRolePrompt

Here we have our `system`, `user`, and `assistant` role prompts.

Let's take a peek at what they look like:

```python
class BasePrompt:
    def __init__(self, prompt):
        """
        Initializes the BasePrompt object with a prompt template.

        :param prompt: A string that can contain placeholders within curly braces
        """
        self.prompt = prompt
        self._pattern = re.compile(r"\{([^}]+)\}")

    def format_prompt(self, **kwargs):
        """
        Formats the prompt string using the keyword arguments provided.

        :param kwargs: The values to substitute into the prompt string
        :return: The formatted prompt string
        """
        matches = self._pattern.findall(self.prompt)
        return self.prompt.format(**{match: kwargs.get(match, "") for match in matches})

    def get_input_variables(self):
        """
        Gets the list of input variable names from the prompt string.

        :return: List of input variable names
        """
        return self._pattern.findall(self.prompt)
```

Then we have our `RolePrompt` which laser focuses us on the role pattern found in most API endpoints for LLMs.

```python
class RolePrompt(BasePrompt):
    def __init__(self, prompt, role: str):
        """
        Initializes the RolePrompt object with a prompt template and a role.

        :param prompt: A string that can contain placeholders within curly braces
        :param role: The role for the message ('system', 'user', or 'assistant')
        """
        super().__init__(prompt)
        self.role = role

    def create_message(self, **kwargs):
        """
        Creates a message dictionary with a role and a formatted message.

        :param kwargs: The values to substitute into the prompt string
        :return: Dictionary containing the role and the formatted message
        """
        return {"role": self.role, "content": self.format_prompt(**kwargs)}
```

We'll look at how the `SystemRolePrompt` is constructed to get a better idea of how that extension works:

```python
class SystemRolePrompt(RolePrompt):
    def __init__(self, prompt: str):
        super().__init__(prompt, "system")
```

That pattern is repeated for our `UserRolePrompt` and our `AssistantRolePrompt` as well.

##### ChatOpenAI

Next we have our model, which is converted to a format analagous to libraries like LangChain and LlamaIndex.

Let's take a peek at how that is constructed:

```python
class ChatOpenAI:
    def __init__(self, model_name: str = "gpt-4.1-mini"):
        self.model_name = model_name
        self.openai_api_key = os.getenv("OPENAI_API_KEY")
        if self.openai_api_key is None:
            raise ValueError("OPENAI_API_KEY is not set")

    def run(self, messages, text_only: bool = True):
        if not isinstance(messages, list):
            raise ValueError("messages must be a list")

        openai.api_key = self.openai_api_key
        response = openai.ChatCompletion.create(
            model=self.model_name, messages=messages
        )

        if text_only:
            return response.choices[0].message.content

        return response
```

#### ❓ Question #3:

When calling the OpenAI API - are there any ways we can achieve more reproducible outputs?

> NOTE: Check out [this section](https://platform.openai.com/docs/guides/text-generation/) of the OpenAI documentation for the answer!

##### ✅ Answer: Yes we can achieve reproducible outputs by pinning production applications to specific model ID's. This is similar to a seed ID in simulations. This enables us to monitor prompt behavior as we continue to iterate.


### Creating and Prompting OpenAI's `gpt-4.1-mini`!

Let's tie all these together and use it to prompt `gpt-4.1-mini`!

In [ ]:
from aimakerspace.openai_utils.prompts import (
    UserRolePrompt,
    SystemRolePrompt,
    AssistantRolePrompt,
)

from aimakerspace.openai_utils.chatmodel import ChatOpenAI

chat_openai = ChatOpenAI()
user_prompt_template = "{content}"
user_role_prompt = UserRolePrompt(user_prompt_template)
system_prompt_template = (
    "You are an expert in {expertise}, you always answer in a kind way."
)
system_role_prompt = SystemRolePrompt(system_prompt_template)

messages = [
    system_role_prompt.create_message(expertise="Python"),
    user_role_prompt.create_message(
        content="What is the best way to write a loop?"
    ),
]

response = chat_openai.run(messages)

In [ ]:
print(response)

## Task 5: Retrieval Augmented Generation

Now we can create a RAG prompt - which will help our system behave in a way that makes sense!

There is much you could do here, many tweaks and improvements to be made!

In [ ]:
RAG_SYSTEM_TEMPLATE = """You are a knowledgeable assistant that answers questions based strictly on provided context.

Instructions:
- Only answer questions using information from the provided context
- If the context doesn't contain relevant information, respond with "I don't know"
- Be accurate and cite specific parts of the context when possible
- Keep responses {response_style} and {response_length}
- Only use the provided context. Do not use external knowledge.
- Only provide answers when you are confident the context supports your response."""

RAG_USER_TEMPLATE = """Context Information:
{context}

Number of relevant sources found: {context_count}
{similarity_scores}

Question: {user_query}

Please provide your answer based solely on the context above."""

rag_system_prompt = SystemRolePrompt(
    RAG_SYSTEM_TEMPLATE,
    strict=True,
    defaults={
        "response_style": "concise",
        "response_length": "brief"
    }
)

rag_user_prompt = UserRolePrompt(
    RAG_USER_TEMPLATE,
    strict=True,
    defaults={
        "context_count": "",
        "similarity_scores": ""
    }
)

Now we can create our pipeline!

In [ ]:
class RetrievalAugmentedQAPipeline:
    def __init__(self, llm: ChatOpenAI, vector_db_retriever: VectorDatabase, 
                 response_style: str = "detailed", include_scores: bool = False) -> None:
        self.llm = llm
        self.vector_db_retriever = vector_db_retriever
        self.response_style = response_style
        self.include_scores = include_scores

    def run_pipeline(self, user_query: str, k: int = 4, **system_kwargs) -> dict:
        # Retrieve relevant contexts
        context_list = self.vector_db_retriever.search_by_text(user_query, k=k)
        
        context_prompt = ""
        similarity_scores = []
        
        for i, (context, score) in enumerate(context_list, 1):
            context_prompt += f"[Source {i}]: {context}\n\n"
            similarity_scores.append(f"Source {i}: {score:.3f}")
        
        # Create system message with parameters
        system_params = {
            "response_style": self.response_style,
            "response_length": system_kwargs.get("response_length", "detailed")
        }
        
        formatted_system_prompt = rag_system_prompt.create_message(**system_params)
        
        user_params = {
            "user_query": user_query,
            "context": context_prompt.strip(),
            "context_count": len(context_list),
            "similarity_scores": f"Relevance scores: {', '.join(similarity_scores)}" if self.include_scores else ""
        }
        
        formatted_user_prompt = rag_user_prompt.create_message(**user_params)

        return {
            "response": self.llm.run([formatted_system_prompt, formatted_user_prompt]), 
            "context": context_list,
            "context_count": len(context_list),
            "similarity_scores": similarity_scores if self.include_scores else None,
            "prompts_used": {
                "system": formatted_system_prompt,
                "user": formatted_user_prompt
            }
        }

In [ ]:
rag_pipeline = RetrievalAugmentedQAPipeline(
    vector_db_retriever=vector_db,
    llm=chat_openai,
    response_style="detailed",
    include_scores=True
)

result = rag_pipeline.run_pipeline(
    # "What is the 'Michael Eisner Memorial Weak Executive Problem'?",
    "What ways of investing are beneficial?",
    k=3,
    response_length="comprehensive", 
    include_warnings=True,
    confidence_required=True
)

print(f"Response: {result['response']}")
print(f"\nContext Count: {result['context_count']}")
print(f"Similarity Scores: {result['similarity_scores']}")

#### ❓ Question #4:

What prompting strategies could you use to make the LLM have a more thoughtful, detailed response?

What is that strategy called?

> NOTE: You can look through our [OpenAI Responses API](https://colab.research.google.com/drive/14SCfRnp39N7aoOx8ZxadWb0hAqk4lQdL?usp=sharing) notebook for an answer to this question if you get stuck!

##### ✅ Answer: 
Chain-of-Thought (CoT) Prompting, Ask the model to "think step by step" or "show your reasoning". 
Few-Shot Learning, Provide examples of the desired response format.
Role-Based Prompting
Give the model a specific role or expertise.
Iterative Prompting
Ask follow-up questions to get more detail.


One can also use the Responses API to control the reasoning and effort and to provide developer instructions. The reasoning controls how much computational effort is put into an answer and the instructions can help guide the model output.


### 🏗️ Activity #1:

Enhance your RAG application in some way! 

Suggestions are: 

- Allow it to work with PDF files
- Implement a new distance metric
- Add metadata support to the vector database
- Use a different embedding model
- Add the capability to ingest a YouTube link

While these are suggestions, you should feel free to make whatever augmentations you desire! If you shared an idea during Session 1, think about features you might need to incorporate for your use case! 

When you're finished making the augments to your RAG application - vibe check it against the old one - see if you can "feel the improvement"!

> NOTE: These additions might require you to work within the `aimakerspace` library - that's expected!

> NOTE: If you're not sure where to start - ask Cursor (CMD/CTRL+L) to guide you through the changes!

## 🎥 YouTube Integration Demo

Here's how to use the new YouTube functionality in your RAG pipeline!

### 📦 Install Required Package

First, let's make sure the required package is installed:


In [ ]:


# Fixed YouTube API Scraper for Professor Damodaran Videos
import requests
import time
from urllib.parse import quote
import os
from dotenv import load_dotenv


# Load environment variables from .env file
load_dotenv()

# Get API key from environment variables
# api_key = os.getenv("OPENAI_API_KEY")
# YouTube API configuration
YOUTUBE_API_KEY = os.getenv("YOUTUBE_API_KEY")  # Your API key
BASE_URL = "https://www.googleapis.com/youtube/v3"

# Professor Damodaran's known channels and search terms
DAMODARAN_CHANNELS = [
    "UCKn6CPi8h4IG3g4ga5FgKfQ",  # Aswath Damodaran's main channel
]

SEARCH_TERMS = [
    "Aswath Damodaran",
    "Professor Damodaran",
    "Damodaran valuation",
    "Damodaran corporate finance",
    "Damodaran investment",
    "Damodaran DCF",
    "Damodaran risk",
    "Damodaran NYU",
    "Damodaran financial modeling",
    "Damodaran cost of capital"
]

def search_youtube_videos(search_term, max_results=10):
    """Search YouTube for videos using the API"""
    url = f"{BASE_URL}/search"
    params = {
        'part': 'snippet',
        'q': search_term,
        'maxResults': max_results,
        'order': 'relevance',
        'type': 'video',
        'key': YOUTUBE_API_KEY
    }
    
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        return response.json()
    except Exception as e:
        print(f"❌ Error searching for '{search_term}': {e}")
        return None

def get_video_details(video_id):
    """Get detailed information about a video"""
    url = f"{BASE_URL}/videos"
    params = {
        'part': 'snippet,statistics,contentDetails',
        'id': video_id,
        'key': YOUTUBE_API_KEY
    }
    
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        return response.json()
    except Exception as e:
        print(f"❌ Error getting details for video {video_id}: {e}")
        return None

def filter_damodaran_videos(search_results):
    """Filter videos to ensure they're from Professor Damodaran"""
    damodaran_videos = []
    
    for item in search_results.get('items', []):
        # Handle different response formats
        if 'id' in item:
            if isinstance(item['id'], dict) and 'videoId' in item['id']:
                video_id = item['id']['videoId']
            elif isinstance(item['id'], str):
                video_id = item['id']
            else:
                print(f"⚠️ Unexpected video ID format: {item['id']}")
                continue
        else:
            print(f"⚠️ No video ID found in item: {item}")
            continue
            
        title = item['snippet']['title']
        channel_title = item['snippet']['channelTitle']
        description = item['snippet']['description']
        
        # Check if it's Damodaran-related content
        damodaran_keywords = ['damodaran', 'aswath', 'professor damodaran']
        is_damodaran = any(keyword.lower() in title.lower() or 
                          keyword.lower() in channel_title.lower() or 
                          keyword.lower() in description.lower() 
                          for keyword in damodaran_keywords)
        
        if is_damodaran:
            # Get video details for filtering
            details = get_video_details(video_id)
            if details and details.get('items'):
                video_info = details['items'][0]
                duration = video_info['contentDetails']['duration']
                view_count = int(video_info['statistics'].get('viewCount', 0))
                
                # Filter for longer, more educational videos
                if 'PT' in duration and view_count > 1000:  # At least 1000 views
                    damodaran_videos.append({
                        'video_id': video_id,
                        'title': title,
                        'channel': channel_title,
                        'url': f"https://www.youtube.com/watch?v={video_id}",
                        'duration': duration,
                        'views': view_count,
                        'description': description[:200] + "..." if len(description) > 200 else description
                    })
            
            time.sleep(0.1)  # Rate limiting
    
    return damodaran_videos

def scrape_damodaran_videos():
    """Main function to scrape Professor Damodaran's videos"""
    print("🎓 Scraping Professor Damodaran's Videos for RAG Training")
    print("=" * 60)
    
    all_videos = []
    
    for term in SEARCH_TERMS:
        print(f"🔍 Searching for: '{term}'")
        
        # Search for videos
        search_results = search_youtube_videos(term, max_results=15)
        if not search_results:
            continue
        
        # Filter for Damodaran videos
        damodaran_videos = filter_damodaran_videos(search_results)
        print(f"  📹 Found {len(damodaran_videos)} relevant videos")
        
        all_videos.extend(damodaran_videos)
        time.sleep(0.5)  # Rate limiting between searches
    
    # Remove duplicates based on video_id
    unique_videos = {}
    for video in all_videos:
        if video['video_id'] not in unique_videos:
            unique_videos[video['video_id']] = video
    
    final_videos = list(unique_videos.values())
    
    print(f"\n📊 Scraping Summary:")
    print(f"  Total unique videos found: {len(final_videos)}")
    
    # Sort by view count (most popular first)
    final_videos.sort(key=lambda x: x['views'], reverse=True)
    
    # Display top videos
    print(f"\n🏆 Top Professor Damodaran Videos for RAG Training:")
    for i, video in enumerate(final_videos[:10], 1):
        print(f"  {i}. {video['title']}")
        print(f"     📺 {video['url']}")
        print(f"     👀 {video['views']:,} views | ⏱️ {video['duration']}")
        print(f"     📝 {video['description']}")
        print()
    
    return final_videos

# Run the scraper
damodaran_videos = scrape_damodaran_videos()

# Extract URLs for the RAG system
if damodaran_videos:
    video_urls = [video['url'] for video in damodaran_videos]
    print(f"✅ Ready to process {len(video_urls)} Professor Damodaran videos!")
    print(f"📋 Video URLs extracted for RAG training")
    
    # Show first few URLs
    print(f"\n🔗 First 5 video URLs:")
    for i, url in enumerate(video_urls[:5], 1):
        print(f"  {i}. {url}")
    
    if len(video_urls) > 5:
        print(f"  ... and {len(video_urls) - 5} more videos")
else:
    print("❌ No Professor Damodaran videos found")
    video_urls = []


In [ ]:
# Check if youtube_transcript_api is available
try:
    import youtube_transcript_api
    print("✅ youtube_transcript_api is available!")
except ImportError:
    print("❌ youtube_transcript_api not found. Please install it:")
    print("   Run in terminal: uv add youtube-transcript-api")
    print("   Or: pip install youtube-transcript-api")
    print("   Then restart your Jupyter kernel")

# Restart kernel and reload modules to fix import issues
import importlib
import sys

# Force reload the text_utils module
if 'aimakerspace.text_utils' in sys.modules:
    importlib.reload(sys.modules['aimakerspace.text_utils'])

try:
    from aimakerspace.text_utils import YouTubeLoader, MultiYouTubeLoader, CharacterTextSplitter
    from aimakerspace.vectordatabase import VectorDatabase
    import asyncio
    print("✅ All imports successful!")
    
    # Multiple YouTube URLs (add as many as you want)
    youtube_urls = [
        "https://www.youtube.com/watch?v=fmlPJBxzpMo",  # Replace with your videos
        "https://www.youtube.com/watch?v=GzKq_p8IKqI",  # Uncomment to add more
        "https://www.youtube.com/watch?v=JMlAi0B-rlE",  # Uncomment to add more
        "https://www.youtube.com/watch?v=c20_S-QgvsA",
        "https://www.youtube.com/watch?v=znmQ7oMiQrM",
        "https://www.youtube.com/watch?v=Z5chrxMuBoo",
        "https://www.youtube.com/watch?v=c20_S-QgvsA",
        "https://www.youtube.com/watch?v=8vYQpWXQ5hE",
    ]
    
    # Load YouTube transcripts from multiple videos
    print(f"Loading transcripts from {len(youtube_urls)} YouTube video(s)...")
    
    if len(youtube_urls) == 1:
        # Single video (original behavior)
        youtube_loader = YouTubeLoader(youtube_urls[0])
        youtube_documents = youtube_loader.load_documents()
        print(f"✅ Loaded {len(youtube_documents)} document(s) from YouTube")
        print(f"Video ID: {youtube_loader.get_video_info()['video_id']}")
        print(f"First 200 characters: {youtube_documents[0][:200]}...")
    else:
        # Multiple videos (new functionality)
        multi_loader = MultiYouTubeLoader(youtube_urls)
        youtube_documents = multi_loader.load_documents()
        
        # Get summary
        summary = multi_loader.get_summary()
        print(f"✅ Successfully loaded: {summary['successful_videos']}/{summary['total_urls']} videos")
        print(f"📄 Total documents: {summary['total_documents']}")
        
        if summary['successful_videos'] > 0:
            print(f"First 200 characters: {youtube_documents[0][:200]}...")
        
        # Show which videos were successfully loaded
        successful_videos = multi_loader.get_successful_videos()
        print(f"\n📺 Successfully loaded videos:")
        for i, video_info in enumerate(successful_videos, 1):
            print(f"  {i}. {video_info['video_id']} - {video_info['url']}")
        
        # Show failed videos if any
        failed_videos = multi_loader.get_failed_videos()
        if failed_videos:
            print(f"\n❌ Failed to load videos:")
            for failed in failed_videos:
                print(f"  - {failed['url']}: {failed['error']}")
    
except ImportError as e:
    print(f"❌ Import error: {e}")
    print("Please restart your Jupyter kernel: Kernel -> Restart")
    print("If the error persists, the package might not be installed in the right environment.")

In [ ]:
# Split the YouTube transcript into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_youtube_documents = text_splitter.split_texts(youtube_documents)

print(f"📄 Split into {len(split_youtube_documents)} chunks")
print(f"First chunk: {split_youtube_documents[0][:300]}...")


In [ ]:
# Create vector database from YouTube content
print("Creating vector database from YouTube content...")
youtube_vector_db = VectorDatabase()
youtube_vector_db = asyncio.run(youtube_vector_db.abuild_from_list(split_youtube_documents))

print("✅ YouTube vector database created!")
print(f"Number of vectors: {len(youtube_vector_db.vectors)}")


In [ ]:
rag_pipeline_youtube = RetrievalAugmentedQAPipeline(
    vector_db_retriever=youtube_vector_db,
    llm=chat_openai,
    response_style="detailed",
    include_scores=True
)

result = rag_pipeline.run_pipeline(
    "What ways of investing are beneficial?",
    k=3,
    response_length="comprehensive", 
    include_warnings=True,
    confidence_required=True
)

print(f"Response: {result['response']}")
print(f"\nContext Count: {result['context_count']}")
print(f"Similarity Scores: {result['similarity_scores']}")          

In [ ]:
# Test the YouTube RAG system
print("🔍 Testing YouTube RAG system...")

# Example query about the video content
query = "What is the main topic discussed in this video?"
results = youtube_vector_db.search_by_text(query, k=3)

print(f"\nQuery: {query}")
print(f"Found {len(results)} relevant chunks:")
for i, (text, similarity) in enumerate(results):
    print(f"\n--- Result {i+1} (similarity: {similarity:.4f}) ---")
    print(text[:200] + "..." if len(text) > 200 else text)


## 🎓 Professor Damodaran Finance Questions Test

Let's test our RAG system with comprehensive finance and investment questions inspired by Professor Damodaran's expertise:


In [ ]:
# Quick Test: Core Professor Damodaran Questions
print("🎯 Quick Test: Core Finance Questions")
print("=" * 50)

# Core questions for quick testing
core_questions = [
    "What ways of investing are beneficial?",  # Your original question
    "What is the DCF model and how do you use it?",
    "How do you calculate the cost of capital?",
    "What are the key principles of valuation?",
    "How do you assess risk in investments?"
]

for i, question in enumerate(core_questions, 1):
    print(f"\n📝 Question {i}: {question}")
    print("─" * 50)
    
    try:
        result = rag_pipeline.run_pipeline(
            question,
            k=2,  # Fewer results for quicker testing
            response_length="concise",  # Shorter responses
            include_warnings=False,
            confidence_required=False
        )
        
        print(f"✅ {result['response'][:200]}...")  # Show first 200 chars
        print(f"📊 Sources: {result['context_count']}")
        
    except Exception as e:
        print(f"❌ Error: {e}")

print(f"\n✅ Quick test completed!")


In [ ]:
# Strategy 1: Use Existing Video URLs (Bypass IP Ban)
print("🎯 Strategy 1: Using Existing Professor Damodaran Video URLs")
print("=" * 60)

import json
import asyncio
from aimakerspace.text_utils import MultiYouTubeLoader, CharacterTextSplitter
from aimakerspace.vectordatabase import VectorDatabase

# Load the existing video URLs
try:
    with open('damodaran_video_urls.json', 'r') as f:
        existing_video_urls = json.load(f)
    
    print(f"✅ Loaded {len(existing_video_urls)} Professor Damodaran video URLs")
    print(f"📺 These URLs were previously scraped and should work")
    
    # Show first few URLs
    print(f"\n🔗 First 10 video URLs:")
    for i, url in enumerate(existing_video_urls[:10], 1):
        print(f"  {i}. {url}")
    
    if len(existing_video_urls) > 10:
        print(f"  ... and {len(existing_video_urls) - 10} more videos")
    
    # Process a subset to avoid overwhelming the system
    print(f"\n🔄 Processing first 20 videos to create knowledge base...")
    subset_urls = existing_video_urls[:20]  # Start with 20 videos
    
    # Load YouTube transcripts from the subset
    multi_loader = MultiYouTubeLoader(subset_urls)
    documents = multi_loader.load_documents()
    
    # Get summary
    summary = multi_loader.get_summary()
    print(f"✅ Successfully loaded: {summary['successful_videos']}/{summary['total_urls']} videos")
    print(f"📄 Total documents: {summary['total_documents']}")
    
    if summary['successful_videos'] > 0:
        # Split into chunks
        print("✂️ Splitting documents into chunks...")
        text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        chunks = text_splitter.split_texts(documents)
        print(f"📄 Created {len(chunks)} chunks")
        
        # Create vector database
        print("🔄 Creating vector database...")
        damodaran_vector_db = VectorDatabase()
        damodaran_vector_db = asyncio.run(damodaran_vector_db.abuild_from_list(chunks))
        
        print(f"✅ Professor Damodaran knowledge base created!")
        print(f"🎯 Vector database contains {len(damodaran_vector_db.vectors)} vectors")
        
        # Test the knowledge base
        print(f"\n🧪 Testing Knowledge Base:")
        test_queries = [
            "What is the DCF model?",
            "How do you calculate the cost of capital?",
            "What are the key principles of valuation?"
        ]
        
        for query in test_queries:
            print(f"\n🔍 Query: {query}")
            results = damodaran_vector_db.search_by_text(query, k=1)
            if results:
                text, similarity = results[0]
                print(f"  📊 Best match (similarity: {similarity:.4f}):")
                print(f"  💡 {text[:200]}...")
            else:
                print(f"  ❌ No results found")
        
        print(f"\n🎉 Strategy 1 Complete!")
        print(f"💡 You now have a working Professor Damodaran knowledge base!")
        
    else:
        print("❌ No videos were successfully processed")
        print("This might be due to IP blocking. Try the other strategies below.")
        
except FileNotFoundError:
    print("❌ damodaran_video_urls.json not found")
    print("Please run the YouTube scraper first or use one of the other strategies below.")
except Exception as e:
    print(f"❌ Error: {e}")
    print("Try the other strategies below.")


In [ ]:
# Strategy 2: Use Proxy/VPN Rotation (Advanced)
print("🎯 Strategy 2: Proxy/VPN Rotation for IP Ban Bypass")
print("=" * 60)

import requests
import time
import random
from urllib.parse import quote

# List of free proxy services (use with caution)
PROXY_LIST = [
    # Add proxy servers here if you have access to them
    # Example: "http://proxy1.example.com:8080"
    # Example: "http://proxy2.example.com:3128"
]

def test_proxy_connection(proxy_url):
    """Test if a proxy is working"""
    try:
        proxies = {"http": proxy_url, "https": proxy_url}
        response = requests.get("https://httpbin.org/ip", proxies=proxies, timeout=10)
        if response.status_code == 200:
            print(f"✅ Proxy {proxy_url} is working")
            return True
    except Exception as e:
        print(f"❌ Proxy {proxy_url} failed: {e}")
    return False

def search_with_proxy_rotation(search_term, max_results=5):
    """Search YouTube with proxy rotation"""
    if not PROXY_LIST:
        print("⚠️ No proxy servers configured")
        print("💡 To use this strategy:")
        print("  1. Get proxy servers from services like ProxyMesh, Bright Data, or free proxy lists")
        print("  2. Add them to the PROXY_LIST above")
        print("  3. Run this cell again")
        return None
    
    for proxy in PROXY_LIST:
        if test_proxy_connection(proxy):
            try:
                proxies = {"http": proxy, "https": proxy}
                url = f"{BASE_URL}/search"
                params = {
                    'part': 'snippet',
                    'q': search_term,
                    'maxResults': max_results,
                    'order': 'relevance',
                    'type': 'video',
                    'key': YOUTUBE_API_KEY
                }
                
                response = requests.get(url, params=params, proxies=proxies, timeout=30)
                response.raise_for_status()
                print(f"✅ Successfully searched with proxy: {proxy}")
                return response.json()
                
            except Exception as e:
                print(f"❌ Search failed with proxy {proxy}: {e}")
                continue
    
    print("❌ All proxies failed")
    return None

# Test proxy rotation (if proxies are configured)
if PROXY_LIST:
    print("🔄 Testing proxy rotation...")
    test_results = search_with_proxy_rotation("Aswath Damodaran", max_results=3)
    if test_results:
        print("✅ Proxy rotation strategy is working!")
    else:
        print("❌ Proxy rotation strategy failed")
else:
    print("💡 Proxy rotation strategy requires proxy servers")
    print("📋 Alternative: Use Strategy 3 (Manual Video Selection) below")


In [ ]:
# Strategy 3: Manual Video Selection (Most Reliable)
print("🎯 Strategy 3: Manual Video Selection (Most Reliable)")
print("=" * 60)

# Curated list of high-quality Professor Damodaran videos
# These are manually selected to ensure they work and contain valuable content
MANUAL_DAMODARAN_VIDEOS = [
    {
        "title": "Session 1: Introduction to Valuation",
        "url": "https://www.youtube.com/watch?v=znmQ7oMiQrM",
        "description": "Lays out the rationale for doing valuation as well as the issues of bias, complexity and uncertainty that bedevil it."
    },
    {
        "title": "Valuation in Four Lessons | Aswath Damodaran | Talks at Google",
        "url": "https://www.youtube.com/watch?v=Z5chrxMuBoo",
        "description": "The tools and practice of valuation is intimidating to most laymen, who assume that they do not have the skills and the capability to value companies."
    },
    {
        "title": "Aswath Damodaran – Laws of Valuation: Revealing the Myths and Misconceptions",
        "url": "https://www.youtube.com/watch?v=c20_S-QgvsA",
        "description": "The Corporate Finance and Valuation Professor shares why companies need to accept that they go through different life cycles."
    },
    {
        "title": "Session 2: The Dark Side of Valuation",
        "url": "https://www.youtube.com/watch?v=8vYQpWXQ5hE",
        "description": "Explores the dark side of valuation, including bias, complexity, and uncertainty."
    },
    {
        "title": "Session 3: The Three Pillars of Value",
        "url": "https://www.youtube.com/watch?v=xV80dt1OZtQ",
        "description": "Discusses the three pillars of value: cash flows, growth, and risk."
    },
    {
        "title": "Session 4: The Cost of Capital",
        "url": "https://www.youtube.com/watch?v=uH-ffKIgb38",
        "description": "Deep dive into calculating the cost of capital for companies."
    },
    {
        "title": "Session 5: The DCF Model",
        "url": "https://www.youtube.com/watch?v=Tdu0QPfoYZQ",
        "description": "Comprehensive explanation of the Discounted Cash Flow (DCF) model."
    },
    {
        "title": "Session 6: Risk and Return",
        "url": "https://www.youtube.com/watch?v=U3D9a_H_Vrs",
        "description": "Understanding risk and return in valuation and investment decisions."
    },
    {
        "title": "Session 7: Growth and Value",
        "url": "https://www.youtube.com/watch?v=LYGYvN5LUbA",
        "description": "The relationship between growth and value in company valuation."
    },
    {
        "title": "Session 8: The Art of Valuation",
        "url": "https://www.youtube.com/watch?v=kyKfJ_7-mdg",
        "description": "The art and science of valuation, including practical applications."
    }
]

print(f"📺 Curated list of {len(MANUAL_DAMODARAN_VIDEOS)} high-quality Professor Damodaran videos:")
print("=" * 60)

for i, video in enumerate(MANUAL_DAMODARAN_VIDEOS, 1):
    print(f"{i}. {video['title']}")
    print(f"   📺 {video['url']}")
    print(f"   📝 {video['description']}")
    print()

# Extract URLs for processing
manual_video_urls = [video['url'] for video in MANUAL_DAMODARAN_VIDEOS]

print(f"🔄 Processing {len(manual_video_urls)} manually selected videos...")
print("💡 These videos are known to work and contain valuable finance content")

# Process the manual videos
try:
    multi_loader = MultiYouTubeLoader(manual_video_urls)
    documents = multi_loader.load_documents()
    
    # Get summary
    summary = multi_loader.get_summary()
    print(f"✅ Successfully loaded: {summary['successful_videos']}/{summary['total_urls']} videos")
    print(f"📄 Total documents: {summary['total_documents']}")
    
    if summary['successful_videos'] > 0:
        # Split into chunks
        print("✂️ Splitting documents into chunks...")
        text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        chunks = text_splitter.split_texts(documents)
        print(f"📄 Created {len(chunks)} chunks")
        
        # Create vector database
        print("🔄 Creating vector database...")
        manual_damodaran_vector_db = VectorDatabase()
        manual_damodaran_vector_db = asyncio.run(manual_damodaran_vector_db.abuild_from_list(chunks))
        
        print(f"✅ Manual Professor Damodaran knowledge base created!")
        print(f"🎯 Vector database contains {len(manual_damodaran_vector_db.vectors)} vectors")
        
        # Test the knowledge base
        print(f"\n🧪 Testing Manual Knowledge Base:")
        test_queries = [
            "What is the DCF model and how do you use it?",
            "How do you calculate the cost of capital?",
            "What are the key principles of valuation?",
            "How do you assess risk in investments?"
        ]
        
        for query in test_queries:
            print(f"\n🔍 Query: {query}")
            results = manual_damodaran_vector_db.search_by_text(query, k=1)
            if results:
                text, similarity = results[0]
                print(f"  📊 Best match (similarity: {similarity:.4f}):")
                print(f"  💡 {text[:200]}...")
            else:
                print(f"  ❌ No results found")
        
        print(f"\n🎉 Strategy 3 Complete!")
        print(f"💡 You now have a reliable Professor Damodaran knowledge base!")
        print(f"📚 This knowledge base is based on manually selected, high-quality videos")
        
        # Show successful videos
        successful_videos = multi_loader.get_successful_videos()
        print(f"\n📺 Successfully processed videos:")
        for i, video_info in enumerate(successful_videos, 1):
            print(f"  {i}. {video_info['video_id']} - {video_info['url']}")
        
    else:
        print("❌ No videos were successfully processed")
        print("This might be due to IP blocking. Try Strategy 4 (Alternative Data Sources) below.")
        
except Exception as e:
    print(f"❌ Error processing manual videos: {e}")
    print("This might be due to IP blocking. Try Strategy 4 (Alternative Data Sources) below.")


In [ ]:
# Strategy 4: Alternative Data Sources (Fallback)
print("🎯 Strategy 4: Alternative Data Sources (Fallback)")
print("=" * 60)

# If YouTube is blocked, we can use alternative sources
print("💡 Alternative approaches when YouTube is blocked:")
print()

print("1. 📚 Use Professor Damodaran's Written Content:")
print("   - His blog posts and articles")
print("   - Academic papers and research")
print("   - Book excerpts and summaries")
print()

print("2. 🎧 Use Audio/Transcript Services:")
print("   - Podcast transcripts")
print("   - Lecture transcripts from universities")
print("   - Conference presentation transcripts")
print()

print("3. 📄 Use Pre-processed Content:")
print("   - Download transcripts manually from YouTube")
print("   - Use browser extensions to extract transcripts")
print("   - Use third-party transcript services")
print()

print("4. 🔄 Use Different Networks:")
print("   - Try from a different internet connection")
print("   - Use mobile hotspot")
print("   - Try from a different location")
print()

print("5. ⏰ Wait and Retry:")
print("   - IP bans are often temporary")
print("   - Wait a few hours and try again")
print("   - Use the existing video URLs you already have")
print()

# Create a simple fallback knowledge base using existing content
print("🔄 Creating Fallback Knowledge Base...")
print("=" * 40)

# Sample Professor Damodaran content (you can expand this)
FALLBACK_DAMODARAN_CONTENT = [
    """
    VIDEO: znmQ7oMiQrM | Session 1: Introduction to Valuation
    
    Valuation is the process of determining the worth of an asset, investment, or company. 
    The key principles of valuation include:
    1. Cash flows are the foundation of value
    2. Growth creates value only when it generates excess returns
    3. Risk affects value through the discount rate
    4. The value of any asset is the present value of its expected cash flows
    
    The three pillars of value are:
    - Cash flows: The money the business generates
    - Growth: How fast those cash flows grow
    - Risk: The uncertainty around those cash flows
    """,
    
    """
    VIDEO: Z5chrxMuBoo | Valuation in Four Lessons
    
    The four lessons of valuation are:
    1. All valuation is biased - we bring our biases to the process
    2. All valuation is uncertain - we cannot predict the future with certainty
    3. All valuation is complex - there are many moving parts
    4. All valuation is contextual - the same company can have different values in different contexts
    
    The DCF (Discounted Cash Flow) model is the most fundamental approach to valuation.
    It involves projecting future cash flows and discounting them back to present value.
    """,
    
    """
    VIDEO: c20_S-QgvsA | Laws of Valuation
    
    The laws of valuation include:
    1. The value of any asset is the present value of its expected cash flows
    2. Growth creates value only when it generates excess returns
    3. Risk affects value through the discount rate
    4. The value of any asset is relative to the alternatives
    
    Cost of capital is the rate of return required by investors to invest in a company.
    It includes the cost of debt and the cost of equity, weighted by their proportions.
    """,
    
    """
    VIDEO: 8vYQpWXQ5hE | The Dark Side of Valuation
    
    The dark side of valuation includes:
    1. Bias - we bring our preconceptions to the process
    2. Complexity - there are many assumptions and inputs
    3. Uncertainty - we cannot predict the future
    4. Manipulation - valuations can be manipulated to support predetermined conclusions
    
    To overcome these challenges, we need to:
    - Be aware of our biases
    - Use multiple approaches
    - Be transparent about assumptions
    - Consider different scenarios
    """,
    
    """
    VIDEO: xV80dt1OZtQ | The Three Pillars of Value
    
    The three pillars of value are:
    1. Cash Flows: The money the business generates
    2. Growth: How fast those cash flows grow
    3. Risk: The uncertainty around those cash flows
    
    Each pillar affects value differently:
    - Higher cash flows increase value
    - Higher growth increases value (if it generates excess returns)
    - Higher risk decreases value (through higher discount rates)
    
    The key is to understand how these three factors interact to determine value.
    """
]

print(f"📚 Created fallback knowledge base with {len(FALLBACK_DAMODARAN_CONTENT)} content pieces")
print("💡 This provides basic Professor Damodaran knowledge even when YouTube is blocked")

# Create vector database from fallback content
try:
    print("🔄 Creating vector database from fallback content...")
    fallback_vector_db = VectorDatabase()
    fallback_vector_db = asyncio.run(fallback_vector_db.abuild_from_list(FALLBACK_DAMODARAN_CONTENT))
    
    print(f"✅ Fallback knowledge base created with {len(fallback_vector_db.vectors)} vectors")
    
    # Test the fallback knowledge base
    print(f"\n🧪 Testing Fallback Knowledge Base:")
    test_queries = [
        "What is the DCF model?",
        "How do you calculate the cost of capital?",
        "What are the key principles of valuation?"
    ]
    
    for query in test_queries:
        print(f"\n🔍 Query: {query}")
        results = fallback_vector_db.search_by_text(query, k=1)
        if results:
            text, similarity = results[0]
            print(f"  📊 Best match (similarity: {similarity:.4f}):")
            print(f"  💡 {text[:200]}...")
        else:
            print(f"  ❌ No results found")
    
    print(f"\n🎉 Strategy 4 Complete!")
    print(f"💡 You now have a fallback Professor Damodaran knowledge base!")
    print(f"📚 This works even when YouTube is blocked")
    
except Exception as e:
    print(f"❌ Error creating fallback knowledge base: {e}")

print(f"\n📋 Summary of IP Ban Workaround Strategies:")
print(f"1. ✅ Use existing video URLs (Strategy 1)")
print(f"2. 🔄 Use proxy/VPN rotation (Strategy 2)")
print(f"3. 🎯 Use manual video selection (Strategy 3)")
print(f"4. 📚 Use alternative data sources (Strategy 4)")
print(f"5. ⏰ Wait and retry later")
print(f"6. 🌐 Try from different network/location")


In [ ]:
# 🎯 Complete IP Ban Workaround Solution
print("🎯 Complete IP Ban Workaround Solution")
print("=" * 60)

# Check which knowledge bases are available
available_knowledge_bases = []

if 'damodaran_vector_db' in locals():
    available_knowledge_bases.append(("damodaran_vector_db", "Strategy 1: Existing URLs"))
    print("✅ Strategy 1: Existing video URLs knowledge base available")

if 'manual_damodaran_vector_db' in locals():
    available_knowledge_bases.append(("manual_damodaran_vector_db", "Strategy 3: Manual Selection"))
    print("✅ Strategy 3: Manual video selection knowledge base available")

if 'fallback_vector_db' in locals():
    available_knowledge_bases.append(("fallback_vector_db", "Strategy 4: Fallback Content"))
    print("✅ Strategy 4: Fallback content knowledge base available")

if not available_knowledge_bases:
    print("❌ No knowledge bases available")
    print("💡 Please run one of the strategy cells above first")
else:
    print(f"\n🎉 You have {len(available_knowledge_bases)} working knowledge base(s)!")
    
    # Create a unified RAG system that uses all available knowledge bases
    def unified_rag_query(question, k=3):
        """
        Unified RAG query that searches all available knowledge bases
        """
        print(f"🔍 Unified RAG Query: {question}")
        print("-" * 50)
        
        all_results = []
        
        # Search your existing vector database (if available)
        if 'vector_db' in locals():
            print("📚 Searching existing knowledge base...")
            existing_results = vector_db.search_by_text(question, k=k)
            for text, similarity in existing_results:
                all_results.append((text, similarity, "Existing Knowledge"))
            print(f"  Found {len(existing_results)} results from existing knowledge base")
        
        # Search all available Professor Damodaran knowledge bases
        for kb_name, kb_description in available_knowledge_bases:
            print(f"🎓 Searching {kb_description}...")
            kb_results = locals()[kb_name].search_by_text(question, k=k)
            for text, similarity in kb_results:
                all_results.append((text, similarity, kb_description))
            print(f"  Found {len(kb_results)} results from {kb_description}")
        
        # Sort by similarity score
        all_results.sort(key=lambda x: x[1], reverse=True)
        
        # Return top results
        top_results = all_results[:k]
        
        print(f"✅ Unified RAG found {len(top_results)} total results")
        return top_results
    
    # Test the unified RAG system
    print(f"\n🧪 Testing Unified RAG System:")
    print("=" * 50)
    
    test_questions = [
        "What is the DCF model and how do you use it?",
        "How do you calculate the cost of capital?",
        "What are the key principles of valuation?",
        "How do you assess risk in investments?"
    ]
    
    for i, question in enumerate(test_questions, 1):
        print(f"\n{'='*70}")
        print(f"📝 Question {i}: {question}")
        print(f"{'='*70}")
        
        # Get unified results
        unified_results = unified_rag_query(question, k=2)
        
        # Display results
        for j, (text, similarity, source) in enumerate(unified_results, 1):
            print(f"\n--- Result {j} (similarity: {similarity:.4f}) from {source} ---")
            
            # Extract video information if available
            if "VIDEO:" in text:
                video_id = text.split("VIDEO:")[1].split("|")[0].strip()
                clean_text = text.split("|", 1)[1].strip() if "|" in text else text
                print(f"📺 From video: {video_id}")
                print(f"💡 Content: {clean_text[:300]}...")
            else:
                print(f"💡 Content: {text[:300]}...")
    
    print(f"\n🎉 Unified RAG System Complete!")
    print(f"💡 Your RAG system now works around IP bans!")
    print(f"📚 You have access to Professor Damodaran's expertise through multiple sources")
    
    # Usage instructions
    print(f"\n📋 How to Use Your IP Ban-Resistant RAG System:")
    print(f"  # For any finance question:")
    print(f"  results = unified_rag_query('your question', k=3)")
    print(f"  # This searches all available knowledge bases")
    print(f"  # Results are ranked by relevance and include source information")
    
    print(f"\n🔧 Available Knowledge Bases:")
    for kb_name, kb_description in available_knowledge_bases:
        print(f"  - {kb_name}: {kb_description}")
    
    print(f"\n💡 Tips for IP Ban Prevention:")
    print(f"  1. Use existing video URLs (Strategy 1) - most reliable")
    print(f"  2. Process videos in small batches")
    print(f"  3. Add delays between requests")
    print(f"  4. Use different networks when possible")
    print(f"  5. Keep fallback content ready (Strategy 4)")

print(f"\n🎯 IP Ban Workaround Complete!")
print(f"🚀 Your RAG system is now resilient to YouTube IP blocking!")
